## Create training datasets for decoder

In [ ]:
import sys
import os
sys.path.append('../../')
import lm_decoder
from utils.lmutils import cer_with_lm_decoder, build_lm_decoder

lmDir = '../../LanguageModels/Typing5gramPunc'

ngramDecoder = build_lm_decoder(
    lmDir,
    max_active=7000,
    min_active=200,
    beam=17,
    lattice_beam=8,
    acoustic_scale=0.23, #lower means weight LM more highly
    ctc_blank_skip_threshold=1.0,
    length_penalty=0.0,
    nbest=100)


In [ ]:

import sys

sys.path.append('../../')
import utils.mat_to_tfrecord_handwriting
from pathlib import Path
import numpy as np
from T18_SessionArgs import get_session_info


sessions = [
    't18.2025.04.01',
     ]

participant = 't18'
bin_compression_factor = 2
channels_to_exclude = list(range(0,0))
channels_to_zero = list(range(0,0)) #[] # leave empty to not zero anything

for session in sessions:
    trials_to_remove, block_nums, num_test_trials = get_session_info(session)
    session_path = str(Path('../../Data', participant, session, 'Handwriting'))
    tfdata_path = str(Path(session_path, 'tfdata_20ms'))

    print(f'Sesison path: {session_path}')
    print(f'tfdata path: {session_path}')
    print('\n')

    args = {
        'session_mat_path': session_path,
        'block_nums': block_nums,
        'num_test_trials': num_test_trials,
        'trials_to_remove': trials_to_remove,
        'channels_to_exclude': channels_to_exclude,
        'channels_to_zero': channels_to_zero,
        'include_thresh_crossings': True,
        'include_spike_power': True,
        'spike_pow_max': 50000,
        'z_score_data': True,
        'global_std': True,
        'bin_compression_factor': bin_compression_factor,
        'save_path': tfdata_path,
    }

    utils.mat_to_tfrecord_handwriting.main(args)

## Train the decoder. Remember to Restart the notebook first!

In [ ]:
import sys
import gc
sys.path.append('../../')
import numpy as np
num_sampled_electrodes = [10,20,30,40,60,80,100,120,140,160,180,200,220,240,260,280,300,320,340,360,380]
random_seeds = np.arange(40)
import importlib
import tensorflow as tf
from omegaconf import OmegaConf
for num_electrodes in num_sampled_electrodes:
    for seed in random_seeds:
        import sys
        import gc
        sys.path.append('../../')
        import numpy as np
        import importlib
        from utils.lmutils import cer_with_lm_decoder
        from omegaconf import OmegaConf
        args = OmegaConf.load('T18_trainArgs.yaml')
        from utils.brainToText_trainDecoder_subsampling import brainToText_decoder
        args['dataset']['nInputFeatures'] = num_electrodes*2
        args['model']['inputLayerSize'] = num_electrodes*2
        args['model']['inputNetwork']['inputLayerSizes'] = [num_electrodes*2]
        sampled_electrodes = np.random.choice(np.arange(384), num_electrodes, replace=False)
        args['dataset']['sampledElectrodes'] = sampled_electrodes.tolist()
        args['loadDir'] = 'null'
        decoder = brainToText_decoder(args)
        infOut, stats = decoder.train()
        args['loadDir'] = 'HandwritingModel'
        decoder = None
        gc.collect()
        decoder = brainToText_decoder(args)
        out, out_by_day = decoder.inference()
        decoder_out = cer_with_lm_decoder(ngramDecoder, out, blankPenalty=0.2, rescore=False)
        print('Num Eelectrodes, Seed: ', str(num_electrodes), seed)
        print('Val CER post LM: ', decoder_out['cer'])
        print('Val WER post LM: ',decoder_out['wer'])
        for d,t in zip(decoder_out['true_transcripts'], decoder_out['decoded_transcripts']):
            print('True :' ,d, ', Decoded: ', t)
        np.save('T18TrainHandwriting-Electrodes-' + str(num_electrodes) + '-Seed-'+str(seed)+'-WERPostLM.npy',decoder_out['all_wer'])
        del decoder
        decoder = None
        gc.collect()
        tf.keras.backend.clear_session()